In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

In [2]:
pwd = os.getcwd()
pwd

'/Users/songsiho/Desktop/4/data_science/project/project'

# Dataset load

In [3]:
prices_df = pd.read_csv('dataset/prices.csv') # 품목별 물가지수
weights_df= pd.read_excel('dataset/weights.xlsx') # 품목별 가중치 
consumer_values_df=pd.read_excel('dataset/consumer_values.xlsx') # 소비자 심리지수
inflation_values_df= pd.read_csv('dataset/inflation.csv',encoding='cp949') # 기대 인플레이션 
consumer_values_details_df=pd.read_csv('dataset/소비자심리지수상세.csv',encoding='cp949') #소비자 심리지수 상세정보


/Users/songsiho/miniforge3/envs/data_science/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# Dataset overview

In [4]:
# consumer_values_details_df.info()

In [5]:
# prices_df.info()

In [6]:
# weights_df.info()

In [7]:
# consumer_values_df.info()

In [8]:
# inflation_values_df.info()

# Dataset preprocessing

## 품목별 물가지수

In [9]:
prices_df.rename(columns = {'주요품목별(1)' : '품목'}, inplace = True)
prices_df.columns = [x.replace('.', '') for x in prices_df.columns]

In [10]:
# prices_df.head()

## 품목별 가중치

In [11]:
weights_df.rename(columns={'2020년 기준 품목 및 가중치':'category','Unnamed: 2':'items','Unnamed: 3':'weight'},inplace=True)
weights_df=weights_df.iloc[2:]
weights_df=weights_df.loc[:,['category','items','weight']]
weights_df = weights_df.reset_index(drop=True)
category_list=(sorted(list(set(weights_df['category']))))
category_dict= {value:index for index,value in enumerate(category_list)}
print(category_dict)
weights_df['category'] = weights_df['category'].apply(lambda x: category_dict[x] if x in category_dict else x)
weights_df

{'01 식료품 및 비주류음료': 0, '02 주류 및 담배': 1, '03 의류 및 신발': 2, '04 주택, 수도, 전기 및 연료': 3, '05 가정용품 및 가사 서비스': 4, '06 보건': 5, '07 교통': 6, '08 통신': 7, '09 오락 및 문화': 8, '10 교육': 9, '11 음식 및 숙박': 10, '12 기타 상품 및 서비스': 11}


,category,items,weight
0,0,쌀,5.5
1,0,현미,0.5
2,0,찹쌀,0.7
3,0,보리쌀,0.2
4,0,콩,0.6
...,...,...,...
453,11,부동산중개수수료,1.7
454,11,행정수수료,0.9
455,11,대입전형료,0.2
456,11,시험응시료,0.4


## 기대 인플레이션

In [12]:
inflation_values_df.columns = [x.replace('.','') for x in inflation_values_df.columns]
inflation_values_df=inflation_values_df[(inflation_values_df['CSI코드별']== '물가인식') | (inflation_values_df['CSI코드별']== '기대인플레이션율')]
#물가인식 : 지난 1년간 소비자물가상승률에 대한 인식 
#기대인플레이션 : 향후 1년간 소비자물가상승률 전망

#소비자의 물가에 대한 인식과 실제 물가 변동률 
#소비자의 물가인식은 최근 몇개월동안의 물가 변동에 영향을 가장 크게 받지 않을까? 

#물가인식과 기대인플레이션율에 대한 연관관계 
#가정 : 물가가 어느정도 선으로 오르면 그정도 오르겠지 라고 생각하는 것 같다 하지만, 특정 임계점 (물가가 너무 올랐다 라고 생각하는 시점) 에서부터는 이것보다는 덜 오르겠지 라는 심리가 반영되고 있는 것 같다

## 소비자 심리지수

In [13]:
consumer_values_df.columns = consumer_values_df.iloc[1]
# consumer_values_df
consumer_values_df=consumer_values_df.iloc[2:3].reset_index(drop=True)
consumer_values_df=consumer_values_df[consumer_values_df.columns[1:]]
consumer_values_df.columns = [x.replace('월','') for x in consumer_values_df.columns]
consumer_values_df


,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,...,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304
0,97.8,80.4,73.3,79.7,83.7,85.9,89.7,81.4,92.9,99.0,...,86.3,89.0,91.6,89.0,86.7,90.2,90.7,90.2,92.0,95.1


## 소비자 심리지수 상세정보

In [14]:
csi_list=sorted(list(set(consumer_values_details_df['CSI코드별'])))
csi_dict = {value: index for index, value in enumerate(csi_list)}
csi_dict
consumer_values_details_df.drop(['항목','단위','Unnamed: 43'],axis=1,inplace=True)
# csi_list
# consumer_values_details_df
consumer_values_details_df['CSI코드별']=consumer_values_details_df['CSI코드별'].apply(lambda x: csi_dict[x] if x in csi_dict else x)
consumer_values_details_df.columns = [i.replace('.',"") for i in consumer_values_details_df.columns]
consumer_values_details_df.columns = [i.replace('월',"") for i in consumer_values_details_df.columns]

In [15]:
# set(consumer_values_details_df['CSI분류코드별'])

In [16]:
# csi_dict

In [17]:
consumer_values_details_df = consumer_values_details_df.sort_values(by='CSI코드별')
consumer_values_details_df.reset_index()

,index,CSI코드별,CSI분류코드별,202002,202003,202004,202005,202006,202007,202008,...,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304
0,396,0,전체,98.0,99.0,102.0,100.0,99.0,100.0,99.0,...,102.0,101,101.0,102,102.0,100.0,102.0,100.0,100,98.0
1,416,0,400-500만원,98.0,96.0,100.0,97.0,99.0,98.0,95.0,...,104.0,99,100.0,99,101.0,103.0,101.0,102.0,103,99.0
2,415,0,70세이상,101.0,102.0,106.0,102.0,99.0,102.0,101.0,...,101.0,101,99.0,99,101.0,100.0,103.0,98.0,100,97.0
3,414,0,300-400만원,102.0,102.0,105.0,101.0,102.0,103.0,102.0,...,106.0,105,105.0,104,108.0,108.0,106.0,106.0,104,104.0
4,413,0,60-70세,97.0,99.0,102.0,100.0,101.0,100.0,98.0,...,101.0,100,96.0,101,99.0,98.0,102.0,97.0,98,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,18,24,300-400만원,87.0,79.0,78.0,78.0,79.0,82.0,82.0,...,79.0,84,84.0,81,80.0,82.0,79.0,78.0,80,83.0
525,19,24,70세이상,84.0,77.0,71.0,77.0,84.0,84.0,81.0,...,77.0,77,81.0,84,82.0,79.0,82.0,80.0,83,84.0
526,21,24,500만원이상,101.0,92.0,86.0,88.0,92.0,94.0,93.0,...,87.0,89,90.0,86,86.0,86.0,87.0,87.0,88,93.0
527,11,24,임차 등,93.0,85.0,81.0,82.0,86.0,88.0,86.0,...,82.0,84,86.0,83,83.0,82.0,81.0,82.0,82,86.0


In [18]:
consumer_values_details_df[consumer_values_details_df['CSI코드별']==0]

,CSI코드별,CSI분류코드별,202002,202003,202004,202005,202006,202007,202008,202009,...,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304
396,0,전체,98.0,99.0,102.0,100.0,99.0,100.0,99.0,101.0,...,102.0,101,101.0,102,102.0,100.0,102.0,100.0,100,98.0
416,0,400-500만원,98.0,96.0,100.0,97.0,99.0,98.0,95.0,98.0,...,104.0,99,100.0,99,101.0,103.0,101.0,102.0,103,99.0
415,0,70세이상,101.0,102.0,106.0,102.0,99.0,102.0,101.0,103.0,...,101.0,101,99.0,99,101.0,100.0,103.0,98.0,100,97.0
414,0,300-400만원,102.0,102.0,105.0,101.0,102.0,103.0,102.0,103.0,...,106.0,105,105.0,104,108.0,108.0,106.0,106.0,104,104.0
413,0,60-70세,97.0,99.0,102.0,100.0,101.0,100.0,98.0,101.0,...,101.0,100,96.0,101,99.0,98.0,102.0,97.0,98,97.0
412,0,기타도시,96.0,97.0,99.0,97.0,98.0,99.0,98.0,101.0,...,102.0,101,100.0,103,102.0,99.0,102.0,101.0,99,97.0
411,0,200-300만원,100.0,104.0,104.0,104.0,102.0,104.0,103.0,108.0,...,103.0,105,104.0,105,104.0,101.0,106.0,102.0,100,101.0
410,0,기타,99.0,101.0,105.0,102.0,101.0,101.0,102.0,103.0,...,101.0,100,97.0,101,102.0,99.0,102.0,99.0,101,97.0
409,0,50-60세,98.0,100.0,105.0,102.0,102.0,100.0,98.0,102.0,...,102.0,100,101.0,102,101.0,98.0,100.0,100.0,100,96.0
417,0,500만원이상,92.0,94.0,96.0,95.0,95.0,97.0,96.0,98.0,...,100.0,98,99.0,100,98.0,96.0,97.0,96.0,96,94.0


In [19]:
# for i in range(0,25):
#     print(len(consumer_values_details_df[consumer_values_details_df['CSI코드별']==i]))

# 데이터 분석
소비자 심리지수와 기대인플레이션 과의 상관관계

In [20]:
consumer_values_details_df.head()

,CSI코드별,CSI분류코드별,202002,202003,202004,202005,202006,202007,202008,202009,...,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304
396,0,전체,98.0,99.0,102.0,100.0,99.0,100.0,99.0,101.0,...,102.0,101,101.0,102,102.0,100.0,102.0,100.0,100,98.0
416,0,400-500만원,98.0,96.0,100.0,97.0,99.0,98.0,95.0,98.0,...,104.0,99,100.0,99,101.0,103.0,101.0,102.0,103,99.0
415,0,70세이상,101.0,102.0,106.0,102.0,99.0,102.0,101.0,103.0,...,101.0,101,99.0,99,101.0,100.0,103.0,98.0,100,97.0
414,0,300-400만원,102.0,102.0,105.0,101.0,102.0,103.0,102.0,103.0,...,106.0,105,105.0,104,108.0,108.0,106.0,106.0,104,104.0
413,0,60-70세,97.0,99.0,102.0,100.0,101.0,100.0,98.0,101.0,...,101.0,100,96.0,101,99.0,98.0,102.0,97.0,98,97.0


### 전체를 기준으로 살펴보기

In [21]:
csi_all=consumer_values_details_df[consumer_values_details_df['CSI분류코드별']=='전체']
csi_all=csi_all[csi_all['CSI코드별'] != 10].sort_values(by='CSI코드별')
csi_all.reset_index(inplace=True)
csi_all.drop('index',axis=1)

,CSI코드별,CSI분류코드별,202002,202003,202004,202005,202006,202007,202008,202009,...,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304
0,0,전체,98.0,99.0,102.0,100.0,99.0,100.0,99.0,101.0,...,102.0,101,101.0,102,102.0,100.0,102.0,100.0,100,98.0
1,1,전체,97.0,87.0,83.0,87.0,88.0,90.0,92.0,88.0,...,93.0,94,96.0,94,93.0,95.0,96.0,95.0,96,96.0
2,2,전체,96.0,90.0,87.0,88.0,91.0,91.0,92.0,89.0,...,88.0,90,92.0,90,91.0,91.0,91.0,91.0,93,95.0
3,3,전체,88.0,77.0,74.0,76.0,77.0,79.0,82.0,74.0,...,93.0,91,93.0,92,89.0,92.0,91.0,91.0,92,93.0
4,4,전체,103.0,95.0,92.0,94.0,95.0,96.0,97.0,93.0,...,97.0,97,99.0,97,96.0,99.0,100.0,101.0,99,99.0
5,5,전체,106.0,99.0,97.0,99.0,100.0,101.0,102.0,99.0,...,114.0,112,110.0,112,110.0,110.0,114.0,117.0,112,111.0
6,6,전체,92.0,72.0,77.0,82.0,82.0,88.0,90.0,89.0,...,152.0,149,147.0,150,151.0,133.0,132.0,113.0,120,111.0
7,7,전체,93.0,85.0,81.0,83.0,84.0,86.0,88.0,83.0,...,89.0,91,92.0,91,89.0,90.0,91.0,91.0,90,93.0
8,8,전체,137.0,134.0,132.0,131.0,132.0,135.0,139.0,139.0,...,166.0,158,157.0,157,156.0,151.0,151.0,153.0,151,148.0
9,9,전체,93.0,83.0,79.0,85.0,87.0,87.0,89.0,85.0,...,79.0,83,86.0,84,82.0,85.0,85.0,83.0,87,90.0


In [22]:
inflation_values_df.head()

,CSI코드별,202002,202003,202004,202005,202006,202007,202008,202009,202010,...,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304
0,물가인식,1.8,1.8,1.8,1.7,1.7,1.7,1.8,1.9,1.9,...,5.1,5.1,5.1,5.2,5.1,5.0,5.0,5.2,5.1,4.9
10,기대인플레이션율,1.7,1.7,1.7,1.6,1.6,1.7,1.8,1.9,1.8,...,4.7,4.3,4.2,4.3,4.2,3.8,3.9,4.0,3.9,3.7
